In [1]:
# Dependencies

import os
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset

c:\Users\Willson\anaconda3\envs\ml\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### ENCODER PART
1. DATA LOADING
2. IMAGE PREPROCESSING, RESIZING, CLEANING
3. VISION TRANSFORMER ENCODER
*   patch embeddding
*   vision transformer
4. TRAINING LOOP

DATA LOADING OF POLYVORE SET
Do we want set to be categorized beforehand or not?

In [2]:
directory = 'c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images'

for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    print(file_path)
    if os.path.isfile(file_path):  # Check if it's a file
        print(file_path)

c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\bag
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\bracelet
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\brooch
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\dress
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\earrings
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\eyewear
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\gloves
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\hairwear
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\hats
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\jumpsuit
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\legwear
c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images\necklace
c:/Users/Willson/Docum

In [2]:
# Split into training, validation sets

import os
import shutil
import random
from pathlib import Path

# Parameters
data_dir = 'c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images'  # The root directory containing your category folders
output_dir = 'c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images_split'   # The directory where the split dataset will be stored
val_split = 0.2                 # Proportion of the dataset to include in the validation split
subset_fraction = 0.1

# Ensure the output directories exist
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Function to split the dataset
def split_dataset(data_dir, train_dir, val_dir, val_split):
    categories = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    
    for category in categories:
        category_path = os.path.join(data_dir, category)
        images = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]
        
        # Create subset
        subset_size = int(len(images) * subset_fraction)
        subset_images = random.sample(images, subset_size)

        # Split into training and validation sets
        val_size = int(len(subset_images) * val_split)
        train_images = subset_images[val_size:]
        val_images = subset_images[:val_size]
        
        train_category_dir = os.path.join(train_dir, category)
        val_category_dir = os.path.join(val_dir, category)
        
        os.makedirs(train_category_dir, exist_ok=True)
        os.makedirs(val_category_dir, exist_ok=True)
        
        for img in train_images:
            shutil.copy2(os.path.join(category_path, img), os.path.join(train_category_dir, img))
        
        for img in val_images:
            shutil.copy2(os.path.join(category_path, img), os.path.join(val_category_dir, img))
        
        print(f'Category: {category}')
        print(f'Total images: {len(images)}')
        print(f'Training images: {len(train_images)}')
        print(f'Validation images: {len(val_images)}')

# Execute the split
split_dataset(data_dir, train_dir, val_dir, val_split)


KeyboardInterrupt: 

In [88]:
# Reduce the number of samples for faster training
import shutil
import random

# Parameters
data_dir = 'c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images_split'  # The root directory containing your current train/val folders
output_dir = 'c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images_split_smaller'  # The directory where the further subset dataset will be stored
subset_fraction = 0.1              # Fraction of the dataset to include in the further subset
min_samples = 10

# Function to create a further subset of the existing training and validation sets
def create_further_subset(data_dir, output_dir, subset_fraction):
    # Define paths for current training and validation data
    current_train_dir = os.path.join(data_dir, 'train')
    current_val_dir = os.path.join(data_dir, 'val')

    # Define paths for new training and validation data
    new_train_dir = os.path.join(output_dir, 'train')
    new_val_dir = os.path.join(output_dir, 'val')

    os.makedirs(new_train_dir, exist_ok=True)
    os.makedirs(new_val_dir, exist_ok=True)

    # Function to process each category
    def process_category(category, current_dir, new_dir):
        category_path = os.path.join(current_dir, category)
        if not os.path.isdir(category_path):
            return
        
        images = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]
        
        # Create further subset
        subset_size = max(int(len(images) * subset_fraction), min_samples)
        subset_size = min(subset_size, len(images))  # Ensure we don't select more images than available
        subset_images = random.sample(images, subset_size)

        new_category_dir = os.path.join(new_dir, category)
        os.makedirs(new_category_dir, exist_ok=True)

        for img in subset_images:
            shutil.copy2(os.path.join(category_path, img), os.path.join(new_category_dir, img))
        
        print(f'Category: {category}')
        print(f'Total images: {len(images)}')
        print(f'Subset images: {len(subset_images)}')

    # Process all categories in both training and validation sets
    categories = os.listdir(current_train_dir)
    for category in categories:
        process_category(category, current_train_dir, new_train_dir)
    
    categories = os.listdir(current_val_dir)
    for category in categories:
        process_category(category, current_val_dir, new_val_dir)

# Execute the further subset creation
create_further_subset(data_dir, output_dir, subset_fraction)

Category: bag
Total images: 1702
Subset images: 170
Category: bracelet
Total images: 414
Subset images: 41
Category: brooch
Total images: 80
Subset images: 10
Category: dress
Total images: 599
Subset images: 59
Category: earrings
Total images: 440
Subset images: 44
Category: eyewear
Total images: 535
Subset images: 53
Category: gloves
Total images: 31
Subset images: 10
Category: hairwear
Total images: 56
Subset images: 10
Category: hats
Total images: 233
Subset images: 23
Category: jumpsuit
Total images: 24
Subset images: 10
Category: legwear
Total images: 16
Subset images: 10
Category: necklace
Total images: 373
Subset images: 37
Category: neckwear
Total images: 96
Subset images: 10
Category: outwear
Total images: 814
Subset images: 81
Category: pants
Total images: 716
Subset images: 71
Category: rings
Total images: 258
Subset images: 25
Category: shoes
Total images: 1611
Subset images: 161
Category: skirt
Total images: 424
Subset images: 42
Category: top
Total images: 1552
Subset ima

IMAGE PREPROCESSING

Resizing down to 256x256 size, or maybe smaller, depending on training speed and quality

VISION TRANSFORMER ENCODER - pre trained model

In [22]:
from transformers import ViTForImageClassification, ViTFeatureExtractor, TrainingArguments, Trainer

# Load pre-trained ViT, feature extractor
model_name = "google/vit-base-patch16-224"
model = ViTForImageClassification.from_pretrained(model_name)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)

# !!!! On top of pre-trained, add another layer for our n labels
num_classes = len(os.listdir(directory))
model.config.num_labels = num_classes
model.classifier = torch.nn.Linear(model.config.hidden_size, num_classes)
# print(model.config)

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)
model.to(device)

Using device: cuda


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

# TRAINING V1 - IGNORE

Output to 1000 classes as pre-trained, did not adjust for our dataset

In [10]:
# Hyperparameters
batch_size = 32
num_epochs = 5
learning_rate = 5e-5

# Data processing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets, DataLoaders
data_dir = 'c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images_split_smaller'
train_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'val'), transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Convert for transformers
class ImageDataset(Dataset):
    def __init__(self, image_folder):
        self.image_folder = image_folder
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.image_folder)

    def __getitem__(self, idx):
        image, label = self.image_folder[idx]
        image = self.feature_extractor(images=image, return_tensors="pt")['pixel_values'][0]
        return {'pixel_values': image, 'labels': label}

train_dataset = ImageDataset(train_dataset)
val_dataset = ImageDataset(val_dataset)

In [11]:
# Train
training_args = TrainingArguments(
    output_dir=os.path.join(data_dir, 'results'),
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    eval_strategy="epoch",
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=feature_extractor
)

trainer.train()

model.save_pretrained('devito')
feature_extractor.save_pretrained('devito')

  0%|          | 0/330 [02:09<?, ?it/s]


RuntimeError: shape '[-1, 1000]' is invalid for input of size 640

In [19]:
# Evaluation
from PIL import Image

# Load our model and feature extractor
our_model = ViTForImageClassification.from_pretrained('devito')
our_feature_extractor = ViTFeatureExtractor.from_pretrained('devito')

# Predict
def predict(image_path):
    image = Image.open(image_path).convert('RGB')
    inputs = our_feature_extractor(images=image, return_tensors="pt")
    outputs = our_model(**inputs)
    logits = outputs.logits
    predicted_class_idx = logits.argmax(-1).item()
    return train_dataset.image_folder.classes[predicted_class_idx]

# Predict on a new image
image_path = 'c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images/gloves/5598757_14.jpg'
# predicted_class = predict(image_path)
# print(f'Predicted class: {predicted_class}')

image = Image.open(image_path).convert('RGB')
inputs = our_feature_extractor(images=image, return_tensors="pt")
outputs = our_model(**inputs)
logits = outputs.logits

len(logits[0])

1000

# TRAINING V2

In [9]:
# DEPENDENCIES

import torch
import torch.nn as nn
from transformers import ViTFeatureExtractor, ViTForImageClassification
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from PIL import Image
import os

In [1]:
# MODEL SETUP AND TRAINING

# Set the device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define data transformations
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your dataset using ImageFolder
data_dir = 'c:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images_split/train'
dataset = ImageFolder(data_dir, transform=data_transforms)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))  # 80% for training, 20% for validation
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for training and validation
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Initialize ViT feature extractor and model
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k')

# Modify the model's final classification layer for your number of classes
num_classes = len(dataset.classes)  # Get the number of classes from your dataset
model.classifier = nn.Linear(model.config.hidden_size, num_classes)  # Replace classifier with new Linear layer

# Move model to device
model.to(device)

# Set up optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# Training loop with validation
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.logits, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_dataloader)
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}')

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs.logits, labels)

            val_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs.logits, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_accuracy = correct / total
    val_loss /= len(val_dataloader)
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

# Save the fine-tuned model
torch.save(model.state_dict(), 'fine_tuned_vit_model.pth')

c:\Users\Willson\anaconda3\envs\ml\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Willson\anaconda3\envs\ml\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Willson\anaconda3\envs\ml\Lib\site-packages\transformers\models\vit\modeling_vit.py:253: UserWarning: 1Torch was not compiled with flash attention. (Triggere

Epoch 1/10, Training Loss: 0.9117
Epoch 1/10, Validation Loss: 0.4212, Validation Accuracy: 0.8969
Epoch 2/10, Training Loss: 0.2562
Epoch 2/10, Validation Loss: 0.3510, Validation Accuracy: 0.9067
Epoch 3/10, Training Loss: 0.1575
Epoch 3/10, Validation Loss: 0.2723, Validation Accuracy: 0.9175
Epoch 4/10, Training Loss: 0.0887
Epoch 4/10, Validation Loss: 0.2892, Validation Accuracy: 0.9214
Epoch 5/10, Training Loss: 0.0690
Epoch 5/10, Validation Loss: 0.2967, Validation Accuracy: 0.9224
Epoch 6/10, Training Loss: 0.0487
Epoch 6/10, Validation Loss: 0.3385, Validation Accuracy: 0.9199
Epoch 7/10, Training Loss: 0.0712
Epoch 7/10, Validation Loss: 0.3256, Validation Accuracy: 0.9131
Epoch 8/10, Training Loss: 0.0431
Epoch 8/10, Validation Loss: 0.3477, Validation Accuracy: 0.9096
Epoch 9/10, Training Loss: 0.0465
Epoch 9/10, Validation Loss: 0.3296, Validation Accuracy: 0.9219
Epoch 10/10, Training Loss: 0.0418
Epoch 10/10, Validation Loss: 0.2815, Validation Accuracy: 0.9298


In [32]:
# TESTING THE MODEL

# Load ViT with our last layer
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k')
# feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
num_classes = len(dataset.classes)
model.classifier = nn.Linear(model.config.hidden_size, num_classes)

# Load trained weights
model.load_state_dict(torch.load('fine_tuned_vit_model.pth'))
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Preprocess image
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = data_transforms(image)
    return image.unsqueeze(0)

# Predict
def predict(image_path, model):
    inputs = preprocess_image(image_path).to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(inputs)
        logits = outputs.logits
        predicted_class_idx = torch.argmax(logits, dim=1).item()
    
    # Get the class name
    class_names = dataset.classes
    predicted_class = class_names[predicted_class_idx]
    return predicted_class

# Use from '/val' which our model has never seen
image_path = 'C:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images_split/val/rings/39931073_6.jpg'
predicted_class = predict(image_path, model)
actual_class = os.path.splitext(os.path.dirname(image_path))[0]
print(f'Predicted: {predicted_class} / Actual: {actual_class}')

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted: rings / Actual: C:/Users/Willson/Documents/shouldiwearit/data/clothing/polyvore/pv_images_split/val/rings


### RECOMMENDER PART

VECTOR EXTRACTION

In [ ]:
# Extracting features (latent vectors) from the trained Vision Transformer
def extract_features(model, dataloader):
    model.eval()
    features = []
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            outputs = model.patch_embed(inputs)  # Get patch embeddings
            cls_tokens = model.cls_token.expand(inputs.size(0), -1, -1)
            x = torch.cat((cls_tokens, outputs), dim=1)
            x = x + model.pos_embed
            x = model.pos_drop(x)
            x = model.transformer(x)
            x = model.norm(x[:, 0])
            features.append(x.cpu().numpy())
    return np.concatenate(features)

# Data loader for feature extraction
feature_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=4)
item_features = extract_features(model, feature_loader)

USER PROFILE AND RECOMMENDATION (currently ML, could be upgraded to a deep learning model based on user specifications)

In [ ]:
# Example: User interaction data
user_interactions = {
    'user1': [0, 1, 2],  # Indices of items the user interacted with
    'user2': [3, 4, 5],
    # ...
}

# Create user profiles based on average features of interacted items
user_profiles = {
    user_id: np.mean([item_features[item_id] for item_id in items], axis=0)
    for user_id, items in user_interactions.items()
}

# Recommendation function
from sklearn.metrics.pairwise import cosine_similarity

def recommend(user_id, top_n=10):
    user_vector = user_profiles[user_id].reshape(1, -1)
    similarities = cosine_similarity(user_vector, item_features)
    recommended_items = np.argsort(similarities[0])[-top_n:]
    return recommended_items

# Example usage
user_id = 'user1'
recommended_items = recommend(user_id)
print(f"Recommended items for user {user_id}: {recommended_items}")